In [1]:
import gensim
from nltk.tokenize import word_tokenize
import string
table = str.maketrans(' ', ' ', string.punctuation)
import numpy as np

In [2]:
#imports the data
import pandas as pd
roughDat= pd.read_csv("../../data/raw/Emergent_NAACL2016/emergent/url-versions-2015-06-14-clean.csv")

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('../../data/external/word2vec/GoogleNews-vectors-negative300.bin', binary=True) 

In [ ]:
#removes unnecessary columns.
roughDat = roughDat.drop(columns=['Unnamed: 0', 'claimId', 'articleHeadlineStance'])

In [ ]:
#mylist=roughDat["claimHeadline"].unique().tolist()
#myseclist = roughDat['articleHeadline'].unique().tolist()
mylist=roughDat["claimHeadline"].tolist()
myseclist = roughDat['articleHeadline'].tolist()

In [ ]:
import re
mylist = [re.sub("-", " ", claim) for claim in mylist]
myseclist = [re.sub("-", " ", headline) for headline in myseclist]

In [38]:
mytokenizedlist =[ word_tokenize(i) for i in mylist]
mysectokenizedlist = [word_tokenize(i) for i in myseclist]

In [30]:
import re

In [43]:
Word2VecFeatureC=[]
for i in mytokenizedlist:
    wf = np.ones(300)
    #wf = np.zeros(300)
    for j in i:
        j = re.sub("-", " ", j)
        j = j.lower()
        j = j.translate(table)
        if j in model.vocab:
            wf*=model[j]
        else:
            continue
            #wf+=model[j]
    Word2VecFeatureC.append(wf)   

In [44]:
Word2VecFeatureH=[]
for i in mysectokenizedlist:
    wf = np.ones(300)
    #wf = np.zeros(300)
    for j in i:
        j = re.sub("-", " ", j)
        j = j.lower()
        j = j.translate(table)
        if j in model.vocab:
            wf*=model[j]
        else:
            continue
            #wf+=model[j]
    Word2VecFeatureH.append(wf)   

In [45]:
from sklearn.metrics.pairwise import cosine_similarity

In [46]:
CosineSimilarityF = [cosine_similarity(Word2VecFeatureH[i].reshape(1,300), Word2VecFeatureC[i].reshape(1,300)) for i in range(0,len(Word2VecFeatureH))]

In [47]:
cosine_df = pd.DataFrame({"articleId": roughDat["articleId"], "cosineSim": [x[0][0] for x in CosineSimilarityF]})

In [48]:
cosine_df.to_csv("../../data/processed/features/cosine_feature.csv", index=False)